In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
cols = ['id','candidate','candidate_confidence','relevant_yn','relevant_yn_confidence','sentiment','sentiment_confidence','subject_matter','subject_matter_confidence','candidate_gold','name','relevant_yn_gold','retweet_count','sentiment_gold','subject_matter_gold','text','tweet_coord','tweet_created','tweet_id','tweet_location','user_timezone']

In [4]:
df = pd.read_csv("Sentiment.csv",header=None, names=cols, encoding = "utf-8")

In [5]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
1,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
2,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
3,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
4,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13872 entries, 0 to 13871
Data columns (total 21 columns):
id                           13872 non-null object
candidate                    13776 non-null object
candidate_confidence         13872 non-null object
relevant_yn                  13872 non-null object
relevant_yn_confidence       13872 non-null object
sentiment                    13872 non-null object
sentiment_confidence         13872 non-null object
subject_matter               13546 non-null object
subject_matter_confidence    13872 non-null object
candidate_gold               29 non-null object
name                         13872 non-null object
relevant_yn_gold             33 non-null object
retweet_count                13872 non-null object
sentiment_gold               16 non-null object
subject_matter_gold          19 non-null object
text                         13872 non-null object
tweet_coord                  22 non-null object
tweet_created                13872 non-

In [7]:
df.sentiment.value_counts()

Negative     8493
Neutral      3142
Positive     2236
sentiment       1
Name: sentiment, dtype: int64

In [8]:
df.drop(['id','candidate','candidate_confidence','relevant_yn','relevant_yn_confidence','sentiment_confidence','subject_matter','subject_matter_confidence','candidate_gold','name','relevant_yn_gold','retweet_count','sentiment_gold','subject_matter_gold','tweet_coord','tweet_created','tweet_id','tweet_location','user_timezone'],axis=1,inplace=True)

In [9]:
df['pre_clean_len'] = [len(t) for t in df.text]

In [10]:
df[df.pre_clean_len > 140].head(10)

,sentiment,text,pre_clean_len
36,Negative,RT @AmyMek: The Torched has been passed -&gt;....,143
53,Negative,"RT @STYLEMOM: Here's a question, WHY in the he...",144
62,Negative,RT @AmyMek: Status 👉 Single \n\nI broke up wit...,144
71,Negative,Fox News Had Its Own #GOPDebate Agenda: Promot...,143
95,Negative,RT @KateThomas: This literally couldn't be mor...,144
101,Neutral,RT @akbarjenkins: Maybe the GOP just got Piven...,148
126,Negative,RT @AmyMek: We all owe @realDonaldTrump a huge...,144
141,Negative,"RT @VH1: Honestly, Donald Trump you were ALL t...",141
143,Negative,Trump gloating abt buying GOP+Dem pols &amp;th...,144
203,Negative,"This literally couldn't be more true for me. ""...",143


In [11]:
df.text[36]

'RT @AmyMek: The Torched has been passed -&gt;.@CandyCrowley is no longer the most Famous Political Assassin! Congratulations @megynkelly, U Wi…'

In [12]:
from bs4 import BeautifulSoup
example1 = BeautifulSoup(df.text[36], 'lxml')
print(example1.get_text())

RT @AmyMek: The Torched has been passed ->.@CandyCrowley is no longer the most Famous Political Assassin! Congratulations @megynkelly, U Wi…


In [13]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [14]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [15]:
%%time
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(0,13872):
    if( (i+1)%1000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, 13872 ))                                                                   
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 1000 of 13872 has been processed
Tweets 2000 of 13872 has been processed
Tweets 3000 of 13872 has been processed
Tweets 4000 of 13872 has been processed
Tweets 5000 of 13872 has been processed
Tweets 6000 of 13872 has been processed
Tweets 7000 of 13872 has been processed
Tweets 8000 of 13872 has been processed
Tweets 9000 of 13872 has been processed
Tweets 10000 of 13872 has been processed
Tweets 11000 of 13872 has been processed
Tweets 12000 of 13872 has been processed
Tweets 13000 of 13872 has been processed
CPU times: user 6.17 s, sys: 321 ms, total: 6.49 s
Wall time: 6.54 s


In [16]:
len(clean_tweet_texts)

13872

In [17]:
df.head()

,sentiment,text,pre_clean_len
0,sentiment,text,4
1,Neutral,RT @NancyLeeGrahn: How did everyone feel about...,106
2,Positive,RT @ScottWalker: Didn't catch the full #GOPdeb...,140
3,Neutral,RT @TJMShow: No mention of Tamir Rice and the ...,84
4,Positive,RT @RobGeorge: That Carly Fiorina is trending ...,140


In [18]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['sentiment'] = df.sentiment
clean_df.head()

,text,sentiment
0,text,sentiment
1,rt how did everyone feel about the climate cha...,Neutral
2,rt didn t catch the full gopdebate last night ...,Positive
3,rt no mention of tamir rice and the gopdebate ...,Neutral
4,rt that carly fiorina is trending hours after ...,Positive


In [19]:
clean_df.to_csv('clean_tweet_kaggle.csv',encoding='utf-8')

In [20]:
10000 positive
4000 negative

SyntaxError: invalid syntax (<ipython-input-20-9fe79c01f6a4>, line 1)